In [1]:
%pwd

'c:\\Users\\Nandhakumar\\Documents\\End to End Projects\\solar-radiation-prediction\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\Nandhakumar\\Documents\\End to End Projects\\solar-radiation-prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir : Path
    val_feature_path: str
    val_target_path: str
    trained_model_path: str
    metric_file_name: str

In [5]:
from solarRadiation.constants import *
from solarRadiation.utils.common import read_yaml, create_directories


In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            val_feature_path= config.val_feature_path,
            val_target_path= config.val_target_path,
            trained_model_path= config.trained_model_path,
            metric_file_name= config.metric_file_name
        )

        return model_evaluation_config

In [7]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import pickle
from solarRadiation.logging import logger
import numpy as np
import pandas as pd

[2023-11-28 20:01:58,554: INFO: utils: NumExpr defaulting to 2 threads.]


In [33]:
class ModelEvaluation:
    def __init__(self, config=ModelEvaluationConfig):
        self.config = config

    def evaluate(self):
        model = pickle.load(open(self.config.trained_model_path,'rb'))
        x_val = pickle.load(open(self.config.val_feature_path,'rb'))
        y_val = pickle.load(open(self.config.val_target_path,'rb'))

        prediction = model.predict(x_val)

        model_score = r2_score(y_val, prediction)
        absolute_error = mean_absolute_error(y_val, prediction)
        squared_error = mean_squared_error(y_val, prediction)
        root_squared_error = np.sqrt(mean_squared_error(y_val, prediction))
        logger.info("Model evaluated successfully")

        metrics_dict = {'Score': [model_score],'MAE':[absolute_error],'MSE':[squared_error],'RMSE':[root_squared_error]}

        df = pd.DataFrame.from_dict(metrics_dict)
        
        df.to_csv(self.config.metric_file_name, index=False)
        logger.info(f"Metrics recorded and stored in {self.config.metric_file_name}")

In [34]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise e

[2023-11-28 20:30:07,270: INFO: common: yaml file config\config.yaml loaded successfully]
[2023-11-28 20:30:07,278: INFO: common: yaml file params.yaml loaded successfully]
[2023-11-28 20:30:07,278: INFO: common: Created directory at artifacts]
[2023-11-28 20:30:07,286: INFO: common: Created directory at artifacts/model_evaluation]
[2023-11-28 20:30:12,430: INFO: 1748672834: Model evaluated successfully]
[2023-11-28 20:30:12,464: INFO: 1748672834: Metrics recorded and stored in artifacts/model_evaluation/metrics.csv]
